In [12]:
import os
import json

from credentials import OPENAPI_KEY
os.environ["OPENAI_API_KEY"] = OPENAPI_KEY

import openai
from tqdm import tqdm
import pandas as pd

In [13]:
client = openai.OpenAI()

In [14]:
ROOT_PATH = "PATH"
ds_name = "HSD2LANG"
ds_path = f"{ROOT_PATH}/{ds_name}.jsonl"
with open(ds_path, "r") as f:
    my_dataset = [json.loads(line) for line in f]

In [15]:
my_dataset[0]

{'messages': [{'role': 'system',
   'content': "VRL-Detector, verilen Türkçe metnin Nefret Söylemi içerip içermediğini tespit edebilen bir chatbot'tur. (Evet, Hayır)"},
  {'role': 'user',
   'content': 'Bu Türkçe metin nefret söylemi içeriyor mu?:" #Yıkılasınisrail https://t.co/Q002Qk4vWn" (Evet, Hayır)'},
  {'role': 'assistant', 'content': 'Evet'}]}

In [16]:
preds = []
refs  = []

for example in tqdm(my_dataset):
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4-turbo"
        messages=example["messages"][:2]
    )
    
    model_output = response.choices[0].message.content
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output)
    refs.append(ground_truth)

100%|██████████| 8805/8805 [1:36:03<00:00,  1.53it/s]   


In [17]:
output_dir = f"{ROOT_PATH}/outputs"
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv(f"{output_dir}/gpt4o-{ds_name}.csv", index=False)

In [18]:
output_df["preds"].value_counts()

preds
Evet.                                              4657
Hayır.                                             2425
Evet                                               1315
Hayır                                               403
Bu metin nefret söylemi içermemektedir. (Hayır)       2
Evet, içeriyor.                                       2
(Evet)                                                1
Name: count, dtype: int64